# Предобработка данных для обучения Random Forest

### Загрузка данных и первоначальная обработка

In [ ]:
import pandas as pd
import pyarrow.parquet as pq
from collections import Counter

print("Шаг 1: Загрузка данных...")
df_train = pq.read_table("train_ai_comp_final_dp.parquet").to_pandas()

# Подготовка данных
print("Шаг 2: Подготовка данных...")
X = df_train.drop(["sample_ml_new", "feature642", "feature756", "target"], axis=1)
y = df_train["target"]

### Сэмплирование данных

In [ ]:
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

# Применение SMOTE для сэмплирования данных
print("Шаг 3: Применение сэмплирования...")
# Подсчет количества примеров в каждом классе
print("До сэмплирования:", Counter(y))


# Создание пайплайна с oversampling и undersampling
pipeline = Pipeline([
    #('oversample', SMOTE(sampling_strategy=1)),  # Увеличение примеров класса меньшинства до 50% от размера класса большинства
    ('undersample', RandomUnderSampler(sampling_strategy=1.0))  # Уменьшение примеров класса большинства до размера класса меньшинства
])
X_resampled, y_resampled = pipeline.fit_resample(X, y)

print("После сэмплирования: ", Counter(y_resampled))

### Выявление наиболее значимых признаков

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import joblib

print("Шаг 4: Выявление наиболее значимых признаков...")
rf_model = RandomForestClassifier(n_estimators=200, random_state=42)
rf_model.fit(X_resampled, y_resampled)
feature_importances = rf_model.feature_importances_
important_features_indices = feature_importances.argsort()[-100:][::-1]  # Выбираем 100 наиболее значимых признаков

# Использование только наиболее значимых признаков
X_important = X_resampled.iloc[:, important_features_indices]
print(important_features_indices)

# Разделение данных на обучающий и валидационный наборы
print("Шаг 5: Разделение данных на обучающий и валидационный наборы...")
X_train, X_val, y_train, y_val = train_test_split(X_important, y_resampled, test_size=0.25, random_state=42)

# Сохранение модели
print("Шаг 6: Сохранение модели...")
joblib.dump(rf_model, 'random_forest_model.model')

### Обучение модели Random Forest

In [ ]:
# Инициализация и обучение модели Random Forest
print("Шаг 7: Инициализация и обучение модели Random Forest...")
rf_model = RandomForestClassifier(n_estimators=300, random_state=42)
rf_model.fit(X_train, y_train)

# Предсказание на валидационном наборе
print("Шаг 8: Предсказание на валидационном наборе...")
y_pred = rf_model.predict(X_val)

In [ ]:
# Вычисление метрик
print("Шаг 9: Вычисление метрик...")
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)
roc_auc = roc_auc_score(y_val, y_pred)

# Сохранение модели
print("Шаг 10: Сохранение новой модели...")
joblib.dump(rf_model, 'random_forest_model_after.model')

# Вывод результатов
print("Метрики:")
print(f"  Precision: {precision}")
print(f"  Recall: {recall}")
print(f"  F1 Score: {f1}")
print(f"  ROC AUC Score: {roc_auc}")

# Проверка модели

In [ ]:
import joblib
import pickle

# Загрузка модели
loaded_model = joblib.load('random_forest_model_after.model')
print(type(loaded_model))
# Загрузка тестовых данных
file_path = "test_sber.parquet"
file_name, file_extension = os.path.splitext(file_path)

needToDelete = ["feature642", "feature756", "sample_ml_new"]

df = pq.read_table(file_path).to_pandas()

df = df.drop(needToDelete, axis=1)

#feature_indices = [1, 4, 43, 67, 88, 89, 90, 91, 93, 96, 98, 119, 124, 129, 130, 137, 195, 262, 263, 270, 282, 283, 284, 286, 287, 288, 309, 313, 342, 349, 350, 357, 358, 359, 385, 395, 396, 402, 432, 434, 437, 438, 440, 441, 445, 461, 464, 467, 472, 479, 481, 482, 485, 489, 529, 548, 614, 667, 674, 675, 692, 720, 726, 738, 741, 744, 745, 752, 753, 780, 791, 797, 808, 810, 813, 852, 859, 862, 863, 852, 859, 862, 863, 852, 859, 862, 863, 852, 859, 862, 863, 852, 859, 862, 863, 852, 859, 862, 863, 852, 859, 862, 863, 852, 859, 862, 863, 986, 992, 996, 998, 1000, 1003, 1004, 1074, 1076, 116, 341, 443, 444, ]
feature_names = loaded_model.feature_names_in_
#feature_names = np.append(feature_names, "id")

print("Идет удаление ненужных признаков...")
df_test = df[feature_names]
print("Удаление завершено\n")

X_test = df

dtest = xgb.DMatrix(X_test)

# Предсказание
predictions = loaded_model.predict_proba(df_test)[:, 1]

rounded_predictions = [round(pred) for pred in predictions]

count_of_1 = sum(predictions)
print("Количество 1: ", count_of_1)
print("Количество всего: ", len(predictions))

# Создание DataFrame с предсказаниями и id
df_predictions = pd.DataFrame({'id': df['id'],'target_bin': rounded_predictions, 'target_prob': predictions})

# Сохранение предсказаний в CSV файл
df_predictions.to_csv('test.csv', index=False)
